# Example for running HiggsSignals to constrain the Higgs charm Yukawa coupling

In [ ]:
import Higgs.predictions as HP
import Higgs.signals as HS
import numpy as np
import pandas as pd

pred = HP.Predictions() # create the model predictions
signals = HS.Signals('/Path/To/HSDataSet') # load HS dataset

### HiggsPredictions input

In [ ]:
# add a SM-like particle
h = pred.addParticle(HP.NeutralScalar("h", "even"))
h.setMass(125.09)

# initalize effC object
cpls = HP.NeutralEffectiveCouplings()
cpls.tt = 1
cpls.bb = 1
cpls.tautau = 1
cpls.ss = 1
cpls.mumu = 1
cpls.gg = 1
cpls.ZZ = 1
cpls.WW = 1
cpls.gamgam = 1
cpls.Zgam = 1

In [ ]:
def setEffC(ccRe, ccIm, refModel):
    cpls.cc = ccRe + 1j * ccIm
    HP.effectiveCouplingInput(h, cpls, reference = refModel)

### Run HiggsSignals

setup scanning ranges

In [ ]:
df = pd.DataFrame()

N = 100

x = np.linspace(-3, 3, N)
y = np.linspace(-3, 3, N)
X, Y = np.meshgrid(x, y)

df['ccRe'] = X.flatten()
df['ccIm'] = Y.flatten()

derive HS results

In [ ]:
@np.vectorize
def calcChisq(ccRe, ccIm, refModel):
    setEffC(ccRe, ccIm, refModel)
    return signals(pred)

In [ ]:
df['chisqSMHiggs']   = calcChisq(df['ccRe'], df['ccIm'], "SMHiggs")
df['chisqSMHiggsEW'] = calcChisq(df['ccRe'], df['ccIm'], "SMHiggsEW")

calculate $\Delta\chi^2$ and cut-off all values above $14.01$ for plotting

In [ ]:
df['dchisqSMHiggs']   = df['chisqSMHiggs'] - df['chisqSMHiggs'].min()
df['dchisqSMHiggsEW'] = df['chisqSMHiggsEW'] - df['chisqSMHiggsEW'].min()

bound = 14.01

@np.vectorize
def cutChisq(e):
    if e < bound:
        return e
    else:
        return bound

df['dchisqSMHiggs']   = cutChisq(df['dchisqSMHiggs'])
df['dchisqSMHiggsEW'] = cutChisq(df['dchisqSMHiggsEW'])

### Plotting

In [ ]:
from scipy import interpolate
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from matplotlib.lines import Line2D
from mpl_toolkits.axes_grid1 import make_axes_locatable

mpl.rcParams['ytick.right'] = True
mpl.rcParams['xtick.top'] = True
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'
mpl.rcParams['text.usetex'] = True
mpl.rcParams['font.family'] = 'serif'

In [ ]:
fig, axs = plt.subplots(
        nrows=1, ncols=2,
        figsize=(1.0 * 6.8, 1.16 * 3.4),
#       aspect='equal',
        constrained_layout=True)

annot = [r'$\mathrm{SMHiggsEW}$', r'$\mathrm{SMHiggs}$']
ids = ['dchisqSMHiggsEW', 'dchisqSMHiggs']

for i, id in enumerate(ids):
        Z = interpolate.griddata( (df['ccRe'], df['ccIm']), df[id], (X, Y), method='linear')

        pcm = axs[i].pcolormesh(
                X, Y, Z,
                cmap='GnBu_r',
                rasterized=True
        )

        the_divider = make_axes_locatable(axs[i])
        cax = the_divider.append_axes("top", size="3.4%", pad=0.1)
        clb = fig.colorbar(
            pcm,
            ticks=np.arange(0, bound, 1),
            cax=cax,
            orientation="horizontal"
        )
        cax.xaxis.set_ticks_position("top")
        cax.xaxis.set_label_position("top")
        clb.set_label(r"$\Delta\chi^2$")
        clb.ax.minorticks_on()

        axs[i].contour(
            X, Y, Z,
            levels=[2.3, 5.99, 9.21],
            colors="k",
            linestyles=["-", "--", ":"],
        )

        bfp = np.unravel_index( np.argmin(Z), Z.shape)
        axs[i].plot( x[bfp[1]], y[bfp[0]], marker="*", ls="none", c="orange")
        
        axs[i].set_aspect('equal', 'box')
        axs[i].set_xlim([-3, 3])
        axs[i].set_ylim([-3, 3])

        percent = "\%" if plt.rcParams["text.usetex"] else "%"
        hsLegend = axs[i].legend(
            handles=[
                Line2D([0], [0], color="k", ls="-", label=f"HS 68{percent} CL"),
                Line2D([0], [0], color="k", ls="--", label=f"HS 95{percent} CL"),
                Line2D([0], [0], color="k", ls=":", label=f"HS 99{percent} CL"),
                Line2D([0], [0], color="orange", ls="none", marker="*", label="HS BFP"),
            ],
            loc="upper left",
            frameon=False,
            ncol=2
        )
        axs[i].add_artist(hsLegend)

        axs[i].xaxis.set_major_locator(MultipleLocator(1))
        axs[i].xaxis.set_minor_locator(MultipleLocator(0.1))
        axs[i].yaxis.set_major_locator(MultipleLocator(1))
        axs[i].yaxis.set_minor_locator(MultipleLocator(0.1))

        axs[i].set_xlabel(r'$c_c$')
        axs[i].set_ylabel(r'$\tilde c_c$')

        axs[i].text(
            0.97,
            0.03,
            annot[i],
            horizontalalignment='right',
            verticalalignment='bottom',
            transform=axs[i].transAxes,
        )

        axs[i].text(
            0.03,
            0.03,
            r"\texttt{HiggsSignals}"
            if plt.rcParams["text.usetex"]
            else r"$\mathrm{HiggsSignals}$",
            horizontalalignment="left",
            verticalalignment="bottom",
            transform=axs[i].transAxes,
        )

plt.savefig('cc.pdf')
plt.show()